In [1]:
import requests # HTTP 호출 패키지
import json # json 형식의 파일 관련 패키지
import pandas as pd # 데이터 분석 및 연산 패키지

### 2. KOSDAQ 오늘자 종목 데이터 수집 및 DBMS에 저장하는 코드 작성 및 제출

In [2]:
# headers 선언
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
    AppleWebKit/537.36 (KHTML, like Gecko)\
    Chrome/100.0.4896.57 Whale/3.14.133.23 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

In [3]:
# payload 데이터 설정 (데이터 속성)

p_data = {
    'bld':'dbms/MDC/STAT/standard/MDCSTAT00601',
    'locale': 'ko_KR',
    'tboxindIdx_finder_equidx0_0': '코스피',
    'indIdx': '2',
    'indIdx2': '001',
    'codeNmindIdx_finder_equidx0_0': '코스피',
    'param1indIdx_finder_equidx0_0': '',
    'trdDd': '20220419',
    'money': '3',
    'csvxls_isNo': 'false'
}

In [4]:
# 데이터의 url 설정
# header-general-Request URL 확인

url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

In [5]:
# HTTP 서버에 데이터 요청
# header-general-Request Method 확인

res = requests.post(url, headers=headers, data=p_data)

In [6]:
# 요청한 데이터를 json 형식으로 html_json에 할당

html_json = json.loads(res.content) 

In [7]:
html_json

{'output': [{'ISU_SRT_CD': '247540',
   'ISU_ABBRV': '에코프로비엠',
   'TDD_CLSPRC': '480,000',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '13,000',
   'FLUC_RT': '2.78',
   'MKTCAP': '11,001,343,680,000'},
  {'ISU_SRT_CD': '091990',
   'ISU_ABBRV': '셀트리온헬스케어',
   'TDD_CLSPRC': '61,500',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '1,600',
   'FLUC_RT': '2.67',
   'MKTCAP': '9,536,221,057,500'},
  {'ISU_SRT_CD': '066970',
   'ISU_ABBRV': '엘앤에프',
   'TDD_CLSPRC': '242,700',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '11,300',
   'FLUC_RT': '4.88',
   'MKTCAP': '8,720,117,803,200'},
  {'ISU_SRT_CD': '263750',
   'ISU_ABBRV': '펄어비스',
   'TDD_CLSPRC': '102,400',
   'FLUC_TP_CD': '2',
   'STR_CMP_PRC': '-200',
   'FLUC_RT': '-0.19',
   'MKTCAP': '6,781,081,600,000'},
  {'ISU_SRT_CD': '293490',
   'ISU_ABBRV': '카카오게임즈',
   'TDD_CLSPRC': '65,900',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '1,400',
   'FLUC_RT': '2.17',
   'MKTCAP': '5,178,928,309,700'},
  {'ISU_SRT_CD': '028300',
   'ISU_ABBRV': 'HLB',
   'TD

In [8]:
# json 형태의 결과값을 DataFrame 형식으로 df에 할당

df = pd.json_normalize(html_json['output'])
df.head()

,ISU_SRT_CD,ISU_ABBRV,TDD_CLSPRC,FLUC_TP_CD,STR_CMP_PRC,FLUC_RT,MKTCAP
0,247540,에코프로비엠,"480,000",1,"13,000",2.78,"11,001,343,680,000"
1,091990,셀트리온헬스케어,"61,500",1,"1,600",2.67,"9,536,221,057,500"
2,066970,엘앤에프,"242,700",1,"11,300",4.88,"8,720,117,803,200"
3,263750,펄어비스,"102,400",2,-200,-0.19,"6,781,081,600,000"
4,293490,카카오게임즈,"65,900",1,"1,400",2.17,"5,178,928,309,700"


In [9]:
# 날짜, 종목 코드, 종목 이름, 지수 종류만 수집하도록 데이터를 변경

html_json = html_json['output']

In [10]:
# 빈 DataFrame을 만들어주고 날짜, 종목 코드, 종목 이름을 추가

name_hs = pd.DataFrame()

if len(html_json) > 0:
    name_h = [] # 빈 list 생성
    for i in range(len(html_json)):
        ISU_SRT_CD = html_json[i]['ISU_SRT_CD'] # 해당 row의 종목 코드 추출
        ISU_ABBRV = html_json[i]['ISU_ABBRV'] # 해당 row의 종목 이름 추출
        # 날짜, 종목 코드, 종목 이름을 name_hs에 추가
        name_h.append(('20220419', ISU_SRT_CD, ISU_ABBRV))
        
    # columns 재설정
    name_h = pd.DataFrame(name_h, columns = ['date', 'stock_code', 'stock_name'])
    name_h['exchange'] = '코스닥'
    name_hs = name_hs.append(name_h, ignore_index=True)

else:
    pass

In [11]:
name_h.head()

,date,stock_code,stock_name,exchange
0,20220419,247540,에코프로비엠,코스닥
1,20220419,091990,셀트리온헬스케어,코스닥
2,20220419,066970,엘앤에프,코스닥
3,20220419,263750,펄어비스,코스닥
4,20220419,293490,카카오게임즈,코스닥


---
#### database 연결 및 데이터 저장

In [12]:
import pymysql as pm

In [13]:
# database 연결

conn = pm.connect(host='localhost', port=3306, db='krx_pr',
                 user='root', passwd='abcd123456', autocommit=True)

In [14]:
cursor = conn.cursor()

In [15]:
# krx_pr db에 kosdaq 테이블 생성
create_kosdaq_table = '''CREATE TABLE if NOT EXISTS KOSDAQ(
	DATE VARCHAR(20),
	stock_code VARCHAR(20),
	stock_name VARCHAR(40),
	EXCHANGE VARCHAR(20),
	PRIMARY KEY (stock_code)
)'''

In [16]:
cursor.execute(create_kosdaq_table) # kospi 테이블 생성

0

In [17]:
# name_h에 저장된 kosdaq 데이터를 db에 삽입

for i in range(len(name_h)):
    date = name_h.date.values[i]
    stock_code = name_h.stock_code.values[i]
    stock_name = name_h.stock_name.values[i]
    exchange = name_h.exchange.values[i]
    
    sql = f"replace into kosdaq (date, stock_code, stock_name, exchange)"\
            f"values ('{date}', '{stock_code}', '{stock_name}', '{exchange}')"
    cursor.execute(sql)
    
conn.commit()

In [18]:
# DBMS 저장 확인
sql = 'select * from kosdaq'
df = pd.read_sql(sql, conn)
df

,DATE,stock_code,stock_name,EXCHANGE
0,20220419,000250,삼천당제약,코스닥
1,20220419,000440,중앙에너비스,코스닥
2,20220419,001000,신라섬유,코스닥
3,20220419,001540,안국약품,코스닥
4,20220419,001810,무림SP,코스닥
...,...,...,...,...
1525,20220419,406760,하나금융21호스팩,코스닥
1526,20220419,408920,에스케이증권7호스팩,코스닥
1527,20220419,409570,한국제10호스팩,코스닥
1528,20220419,413600,키움제6호스팩,코스닥


In [19]:
conn.close() # DB 연결 종료